In [1]:
import trainer
from utils import batchify
import json
import tensorflow as tf
import numpy as np

In [2]:
with open('IntentDetection/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('IntentDetection/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
VERSION = 'intent_102'
params = dict(
    model_configs = {
      "rnn_layers":[
          {
             "units": 1024,
             "drop_i": 0.2,
             "wdrop": 0.4,
             "drop_o": 0.2
          },
          {
             "units": 1024,
             "wdrop": 0.4,
             "drop_o": 0.2
          },
          {
             "units": 1024,
             "drop_o": 0.2,
             "wdrop": 0.4
          }
],
       "vocab_size": len(word2idx) + 1,
       "drop_e": 0.0,
       "char_vocab_size": len(char2idx) + 1,
       "char_cnn_options": {
           "layers": [
                [1, 16],
                [2, 16],
                [3, 32],
                [4, 64],
                [5, 128],
                [6, 256],
                [7, 512]
           ],
           "n_highways": 2
       },
       "char_vec_size": 16,
       "projection_dims": 512,
       "skip_connection": True
    },
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 10240,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

# In[12]:


my_trainer = trainer.Trainer(**params, fine_tune=True)

my_trainer.logger.info('Trainer params {}'.format(params))
# In[13]:


tf.reset_default_graph()
my_trainer.build()

Trainer params {'model_configs': {'rnn_layers': [{'units': 1024, 'drop_i': 0.2, 'wdrop': 0.4, 'drop_o': 0.2}, {'units': 1024, 'wdrop': 0.4, 'drop_o': 0.2}, {'units': 1024, 'drop_o': 0.2, 'wdrop': 0.4}], 'vocab_size': 157876, 'drop_e': 0.0, 'char_vocab_size': 1415, 'char_cnn_options': {'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]], 'n_highways': 2}, 'char_vec_size': 16, 'projection_dims': 512, 'skip_connection': True}, 'optimizer': <class 'tensorflow.python.training.gradient_descent.GradientDescentOptimizer'>, 'negative_samples': 10240, 'wdecay': 1.2e-06, 'alpha': 1e-06, 'beta': 1e-06, 'clip_norm': 0.3, 'bptt': 70, 'use_ema': True, 'save_freq': 1000, 'log_path': 'intent_102/logs', 'train_summary_dir': 'intent_102/train_summary/', 'test_summary_dir': 'intent_102/test_summary/', 'checkpoint_dir': 'intent_102/checkpoints/'}


INFO:tensorflow:Restoring parameters from intent_102/checkpoints/train/model.cpkt-1324235


Restoring parameters from intent_102/checkpoints/train/model.cpkt-1324235


In [4]:
with open('IntentDetection/lm_train_word.npy','rb') as inp:
    train_word = np.load(inp)
with open('IntentDetection/lm_train_char.npy','rb') as inp:
    train_char = np.load(inp)
with open('IntentDetection/lm_valid_word.npy','rb') as inp:
    valid_word = np.load(inp)
with open('IntentDetection/lm_valid_char.npy','rb') as inp:
    valid_char = np.load(inp)

In [6]:
train_word = batchify(train_word, 25).T
train_char = batchify(train_char, 25).T
valid_word = batchify(valid_word, 19).T
valid_char = batchify(valid_char, 19).T

In [7]:
# Reset global step to zero
my_trainer.session.run(tf.assign(my_trainer.global_step, 0))

0

In [8]:
start_i = 0
lr = 5.0
fine_tune_rate = [1/2.6, 1/3.6, 1/3.6, 1/2.6]
# fine_tune_rate = [1, 1, 1, 1]
print(fine_tune_rate)
for _ in range(10):
    my_trainer.train_dev_loop(train_word, train_char, valid_word, valid_char, lr, start_i=start_i, folder_test='test_intent', folder_train='train_intent', fine_tune_rate=fine_tune_rate)
    lr = max(lr * 0.75, 0.5)
    start_i = 0

[0.3846153846153846, 0.2777777777777778, 0.2777777777777778, 0.3846153846153846]


Step    1: progress 69/800000,forward loss 5.73163, backward loss 5.86617, ppl 329.94, bpc 08.37, time 09.37
Step    2: progress 137/800000,forward loss 5.26976, backward loss 5.50593, ppl 218.73, bpc 07.77, time 09.92
Step    3: progress 213/800000,forward loss 5.39719, backward loss 4.98437, ppl 179.61, bpc 07.49, time 10.66
Step    4: progress 283/800000,forward loss 5.12532, backward loss 5.06608, ppl 163.32, bpc 07.35, time 11.27
Step    5: progress 349/800000,forward loss 5.28447, backward loss 5.08405, ppl 178.44, bpc 07.48, time 11.87
Step    6: progress 424/800000,forward loss 4.95829, backward loss 4.78120, ppl 130.29, bpc 07.03, time 12.58
Step    7: progress 453/800000,forward loss 4.94102, backward loss 4.45395, ppl 109.67, bpc 06.78, time 12.95
Step    8: progress 529/800000,forward loss 4.59391, backward loss 4.79378, ppl 109.27, bpc 06.77, time 13.42
Step    9: progress 601/800000,forward loss 4.53669, backward loss 4.41696, ppl 87.95, bpc 06.46, time 14.14
Step   10: p

Step   76: progress 5267/800000,forward loss 3.73015, backward loss 3.83059, ppl 43.83, bpc 05.45, time 52.50
Step   77: progress 5312/800000,forward loss 3.48372, backward loss 3.93352, ppl 40.80, bpc 05.35, time 53.12
Step   78: progress 5383/800000,forward loss 3.71540, backward loss 3.62767, ppl 39.31, bpc 05.30, time 53.79
Step   79: progress 5448/800000,forward loss 3.54610, backward loss 3.90595, ppl 41.51, bpc 05.38, time 54.37
Step   80: progress 5523/800000,forward loss 3.82983, backward loss 4.08933, ppl 52.44, bpc 05.71, time 54.84
Step   81: progress 5591/800000,forward loss 3.36898, backward loss 3.67211, ppl 33.80, bpc 05.08, time 55.37
Step   82: progress 5674/800000,forward loss 3.50119, backward loss 3.92104, ppl 40.90, bpc 05.35, time 56.12
Step   83: progress 5739/800000,forward loss 3.24941, backward loss 3.85527, ppl 34.89, bpc 05.12, time 56.65
Step   84: progress 5773/800000,forward loss 3.56247, backward loss 3.89003, ppl 41.52, bpc 05.38, time 57.10
Step   85:

Step  151: progress 10194/800000,forward loss 3.43341, backward loss 3.58652, ppl 33.45, bpc 05.06, time 92.33
Step  152: progress 10260/800000,forward loss 3.44643, backward loss 3.51889, ppl 32.55, bpc 05.02, time 92.77
Step  153: progress 10330/800000,forward loss 3.57888, backward loss 3.37253, ppl 32.32, bpc 05.01, time 93.20
Step  154: progress 10386/800000,forward loss 3.19481, backward loss 3.40454, ppl 27.10, bpc 04.76, time 93.68
Step  155: progress 10457/800000,forward loss 3.18753, backward loss 3.43464, ppl 27.41, bpc 04.78, time 94.16
Step  156: progress 10516/800000,forward loss 3.42719, backward loss 3.07605, ppl 25.83, bpc 04.69, time 94.71
Step  157: progress 10592/800000,forward loss 3.51762, backward loss 3.33985, ppl 30.84, bpc 04.95, time 95.36
Step  158: progress 10665/800000,forward loss 3.40835, backward loss 3.46883, ppl 31.14, bpc 04.96, time 96.00
Step  159: progress 10739/800000,forward loss 3.62020, backward loss 3.49543, ppl 35.09, bpc 05.13, time 96.65
S

Step  225: progress 15009/800000,forward loss 2.82015, backward loss 4.08388, ppl 31.56, bpc 04.98, time 129.99
Step  226: progress 15082/800000,forward loss 3.39638, backward loss 3.21208, ppl 27.23, bpc 04.77, time 130.46
Step  227: progress 15153/800000,forward loss 3.30244, backward loss 3.06141, ppl 24.09, bpc 04.59, time 130.87
Step  228: progress 15216/800000,forward loss 3.23946, backward loss 3.10214, ppl 23.83, bpc 04.57, time 131.27
Step  229: progress 15290/800000,forward loss 3.11507, backward loss 3.02426, ppl 21.53, bpc 04.43, time 131.90
Step  230: progress 15361/800000,forward loss 3.43547, backward loss 2.94741, ppl 24.32, bpc 04.60, time 132.35
Step  231: progress 15427/800000,forward loss 3.53482, backward loss 3.08911, ppl 27.44, bpc 04.78, time 132.90
Step  232: progress 15494/800000,forward loss 3.19938, backward loss 3.11019, ppl 23.45, bpc 04.55, time 133.35
Step  233: progress 15565/800000,forward loss 3.27463, backward loss 3.14785, ppl 24.81, bpc 04.63, time

...

Step 11788: progress 797859/800000,forward loss 2.54099, backward loss 2.26523, ppl 11.06, bpc 03.47, time 5249.51
Step 11789: progress 797929/800000,forward loss 2.47163, backward loss 2.44920, ppl 11.71, bpc 03.55, time 5249.97
Step 11790: progress 797999/800000,forward loss 2.42343, backward loss 2.62937, ppl 12.51, bpc 03.64, time 5250.41
Step 11791: progress 798074/800000,forward loss 2.35378, backward loss 2.40824, ppl 10.82, bpc 03.44, time 5250.87
Step 11792: progress 798145/800000,forward loss 2.67075, backward loss 2.64452, ppl 14.26, bpc 03.83, time 5251.30
Step 11793: progress 798205/800000,forward loss 2.56654, backward loss 2.69035, ppl 13.85, bpc 03.79, time 5251.69
Step 11794: progress 798280/800000,forward loss 2.62246, backward loss 2.97578, ppl 16.43, bpc 04.04, time 5252.14
Step 11795: progress 798352/800000,forward loss 2.50047, backward loss 2.70863, ppl 13.53, bpc 03.76, time 5252.59
Step 11796: progress 798417/800000,forward loss 2.49210, backward loss 2.91203, 

Evaluate loss 3.2539587020874023, time 24.025819301605225
Evaluate loss 3.1515161991119385, time 24.320229291915894
Evaluate loss 3.165775775909424, time 24.611674547195435
Evaluate loss 3.279283285140991, time 24.907376766204834
Evaluate loss 3.3695669174194336, time 25.20155668258667
Evaluate loss 3.4568660259246826, time 25.49133586883545
Evaluate loss 3.3975601196289062, time 25.792436361312866
Evaluate loss 3.3003058433532715, time 26.094237327575684
Evaluate loss 3.3981173038482666, time 26.3809335231781
Evaluate loss 3.3174474239349365, time 26.687637090682983
Evaluate loss 3.2315099239349365, time 26.982192754745483
Evaluate loss 3.0575156211853027, time 27.282475233078003
Evaluate loss 3.320634126663208, time 27.576437950134277
Evaluate loss 3.1788694858551025, time 27.859837532043457
Evaluate loss 3.3252005577087402, time 28.15121579170227
Evaluate loss 3.3094964027404785, time 28.435688972473145
Evaluate loss 3.1857123374938965, time 28.717724561691284
Evaluate loss 3.265867

Evaluate loss 3.5546295642852783, time 66.61425375938416
Evaluate loss 3.4211621284484863, time 66.91183423995972
Evaluate loss 3.4139721393585205, time 67.20724153518677
Evaluate loss 3.226703405380249, time 67.49719333648682
Evaluate loss 3.3640315532684326, time 67.79159665107727
Evaluate loss 3.2553045749664307, time 68.08390760421753
Evaluate loss 3.204226493835449, time 68.38584303855896
Evaluate loss 3.1513922214508057, time 68.67351603507996
Evaluate loss 3.1559336185455322, time 68.9685480594635
Evaluate loss 3.1400177478790283, time 69.26176953315735
Evaluate loss 3.408590078353882, time 69.55770564079285
Evaluate loss 3.2918083667755127, time 69.85378956794739
Evaluate loss 3.3023836612701416, time 70.14187669754028
Evaluate loss 3.3757011890411377, time 70.4311454296112
Evaluate loss 3.252817392349243, time 70.72021675109863
Evaluate loss 3.399826765060425, time 71.01103234291077
Evaluate loss 3.1305043697357178, time 71.30552315711975
Evaluate loss 3.2690417766571045, time

Step 11854: progress 2310/800000,forward loss 2.42341, backward loss 2.56256, ppl 12.10, bpc 03.60, time 14.62
Step 11855: progress 2381/800000,forward loss 2.50858, backward loss 2.63280, ppl 13.07, bpc 03.71, time 15.07
Step 11856: progress 2449/800000,forward loss 2.24271, backward loss 2.73858, ppl 12.07, bpc 03.59, time 15.49
Step 11857: progress 2515/800000,forward loss 2.39898, backward loss 2.60247, ppl 12.19, bpc 03.61, time 15.91
Step 11858: progress 2588/800000,forward loss 2.40186, backward loss 2.76016, ppl 13.21, bpc 03.72, time 16.37
Step 11859: progress 2661/800000,forward loss 2.36494, backward loss 2.47308, ppl 11.23, bpc 03.49, time 16.83
Step 11860: progress 2739/800000,forward loss 2.23887, backward loss 2.46554, ppl 10.51, bpc 03.39, time 17.31
Step 11861: progress 2811/800000,forward loss 2.29877, backward loss 2.79957, ppl 12.80, bpc 03.68, time 17.76
Step 11862: progress 2875/800000,forward loss 2.51650, backward loss 2.70452, ppl 13.61, bpc 03.77, time 18.19
S

KeyboardInterrupt: 

In [ ]:
for _ in range(10):
    my_trainer.train_dev_loop(train_word, train_char, valid_word, valid_char, lr, start_i=start_i, folder_test='test_sa', folder_train='train_sa', fine_tune_rate=fine_tune_rate)
    lr = max(lr * 0.8, 2.0)
    start_i = 0